In [ ]:
import cv2
from ultralytics import YOLO

In [2]:
model = YOLO('yolo11l.pt')

In [3]:
class_names = model.names
class_names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [ ]:
cap = cv2.VideoCapture(r'C:\Users\TADAR TAGAR\tadar\853889-hd_1920_1080_25fps.mp4') # insert the path of video


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True) 
    print(results)

    # Ensure results are not empty
    if results[0].boxes.data is not None:
        # Get the detected boxes, their class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []  # Handle as needed
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        # Loop through each detected object
        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2  
            cy = (y1 + y2) // 2            

            class_name = class_names[class_idx]

            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)           
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 
            
        cv2.imshow('yolo_tracking', frame)
    
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Dictionary to store center points for each track_id
track_history = {}

# Set the maximum number of frames to track
max_frames = 20

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True , classes = [0] )
    
    if results[0].boxes.data is not None:
        # Get detected boxes, class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []  # Handle as needed
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_idx]

            # Update track history
            if track_id not in track_history:
                track_history[track_id] = []
            track_history[track_id].append((cx, cy))
            
            # Keep only the last 20 frames
            if len(track_history[track_id]) > max_frames:
                track_history[track_id].pop(0)
            
            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw the track history
            if len(track_history[track_id]) > 1:
                for i in range(1, len(track_history[track_id])):
                    cv2.line(frame, track_history[track_id][i-1], track_history[track_id][i], (255, 0, 0), 2)

        cv2.imshow('yolo_tracking', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()



0: 384x640 26 persons, 498.4ms
Speed: 7.4ms preprocess, 498.4ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)



In [ ]:
# Dictionary to store center points for each track_id
track_history = {}

# Set the maximum number of frames to track
max_frames = 20

# Counters for total objects and different objects
total_objects = 0
object_type_counts = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True)  # Specify classes if needed
    
    if results[0].boxes.data is not None:
        # Get detected boxes, class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []  # Handle as needed
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_idx]

            # Update total object count
            total_objects += 1

            # Update count for this class type
            if class_name in object_type_counts:
                object_type_counts[class_name] += 1
            else:
                object_type_counts[class_name] = 1

            # Update track history
            if track_id not in track_history:
                track_history[track_id] = []
            track_history[track_id].append((cx, cy))
            
            # Keep only the last 20 frames
            if len(track_history[track_id]) > max_frames:
                track_history[track_id].pop(0)
            
            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw the track history
            if len(track_history[track_id]) > 1:
                for i in range(1, len(track_history[track_id])):
                    cv2.line(frame, track_history[track_id][i-1], track_history[track_id][i], (255, 0, 0), 2)

        # Display counts on the frame
        cv2.putText(frame, f"Total Objects: {total_objects}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y_offset = 50
        for class_name, count in object_type_counts.items():
            cv2.putText(frame, f"{class_name}: {count}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            y_offset += 20

        cv2.imshow('yolo_tracking', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Dictionary to store center points for each track_id
track_history = {}

# Set the maximum number of frames to track
max_frames = 20

# Set to store unique track IDs
unique_track_ids = set()

# Dictionary to count different objects
object_type_counts = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True)  # Specify classes if needed
    
    if results[0].boxes.data is not None:
        # Get detected boxes, class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []  # Handle as needed
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_idx]

            # Update unique track IDs
            if track_id not in unique_track_ids:
                unique_track_ids.add(track_id)

                # Update count for this class type
                if class_name in object_type_counts:
                    object_type_counts[class_name] += 1
                else:
                    object_type_counts[class_name] = 1

            # Update track history
            if track_id not in track_history:
                track_history[track_id] = []
            track_history[track_id].append((cx, cy))
            
            # Keep only the last 20 frames
            if len(track_history[track_id]) > max_frames:
                track_history[track_id].pop(0)
            
            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw the track history
            if len(track_history[track_id]) > 1:
                for i in range(1, len(track_history[track_id])):
                    cv2.line(frame, track_history[track_id][i-1], track_history[track_id][i], (255, 0, 0), 2)

        # Display counts on the frame
        total_objects = len(unique_track_ids)
        cv2.putText(frame, f"Total Objects: {total_objects}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y_offset = 50
        for class_name, count in object_type_counts.items():
            cv2.putText(frame, f"{class_name}: {count}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            y_offset += 20

        cv2.imshow('yolo_tracking', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import time

track_history = {}

# Define the circle parameters (x, y, radius)
circle_center = (1021, 164)  # Example circle center
circle_radius = 100  # Example radius of the circle

# Dictionary to store the time spent in the circle for each track_id
time_in_circle = {}

# Dictionary to store people who are inside the circle
people_in_circle = set()

# Set the maximum number of frames to track
max_frames = 20

# Set to store unique track IDs
unique_track_ids = set()

# Dictionary to count different objects
object_type_counts = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True)  # Specify classes if needed
    
    if results[0].boxes.data is not None:
        # Get detected boxes, class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []  # Handle as needed
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_idx]

            # Update unique track IDs
            if track_id not in unique_track_ids:
                unique_track_ids.add(track_id)

                # Update count for this class type
                if class_name in object_type_counts:
                    object_type_counts[class_name] += 1
                else:
                    object_type_counts[class_name] = 1

            # Update track history
            if track_id not in track_history:
                track_history[track_id] = []
            track_history[track_id].append((cx, cy))

            # Keep only the last 20 frames
            if len(track_history[track_id]) > max_frames:
                track_history[track_id].pop(0)

            # Check if the person is inside the circle
            distance_to_center = ((cx - circle_center[0]) ** 2 + (cy - circle_center[1]) ** 2) ** 0.5
            if distance_to_center <= circle_radius:
                if track_id not in people_in_circle:
                    people_in_circle.add(track_id)
                    time_in_circle[track_id] = time.time()  # Track the entry time for new people

            # If the person is inside the circle, calculate the time spent
            if track_id in people_in_circle:
                time_spent = time.time() - time_in_circle[track_id]
                minutes_spent = time_spent / 60  # Convert seconds to minutes
                cv2.putText(frame, f"Time in Circle: {minutes_spent:.2f} min", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

            # Draw center point and track information
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw the track history
            if len(track_history[track_id]) > 1:
                for i in range(1, len(track_history[track_id])):
                    cv2.line(frame, track_history[track_id][i-1], track_history[track_id][i], (255, 0, 0), 2)

        # Display counts on the frame
        total_objects = len(unique_track_ids)
        cv2.putText(frame, f"Total Objects: {total_objects}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y_offset = 50
        for class_name, count in object_type_counts.items():
            cv2.putText(frame, f"{class_name}: {count}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            y_offset += 20

        # Draw the circle on the frame
        cv2.circle(frame, circle_center, circle_radius, (0, 0, 255), 2)

        cv2.imshow('yolo_tracking', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

# Initialize variables
circle_radius = 100
circle_center = None

# Mouse callback function to set the center of the circle
def set_circle_position(event, x, y, flags, param):
    global circle_center
    if event == cv2.EVENT_LBUTTONDOWN:  # When left mouse button is clicked
        circle_center = (x, y)  # Set the center of the circle to the click position

# Create a window and bind the mouse callback function
cv2.namedWindow('Select Circle Position')
cv2.setMouseCallback('Select Circle Position', set_circle_position)

# Start video capture (or any frame source)
cap = cv2.VideoCapture(0)  # You can replace this with your video stream source

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if circle_center:
        # Draw the circle on the frame at the selected position
        cv2.circle(frame, circle_center, circle_radius, (0, 0, 255), 2)

    # Show the frame with circle
    cv2.imshow('Select Circle Position', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

# Initialize variables
circle_radius = 100
circle_center = None

# Mouse callback function to set the center of the circle
def set_circle_position(event, x, y, flags, param):
    global circle_center
    if event == cv2.EVENT_LBUTTONDOWN:  # When left mouse button is clicked
        circle_center = (x, y)  # Set the center of the circle to the click position
        print(f"Selected Circle Center: {circle_center}")  # Print the selected position

# Create a window and bind the mouse callback function
cv2.namedWindow('Select Circle Position')
cv2.setMouseCallback('Select Circle Position', set_circle_position)

# Start video capture (use a video file instead of the webcam)
video_path = r'C:\Users\TADAR TAGAR\tadar\853889-hd_1920_1080_25fps.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if circle_center:
        # Draw the circle on the frame at the selected position
        cv2.circle(frame, circle_center, circle_radius, (0, 0, 255), 2)

    # Show the frame with circle
    cv2.imshow('Select Circle Position', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Dictionary to store center points for each track_id
track_history = {}

# Set the maximum number of frames to track
max_frames = 20

# Set to store unique track IDs
unique_track_ids = set()

# Dictionary to count different objects
object_type_counts = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True)  # Specify classes if needed
    
    if results[0].boxes.data is not None:
        # Get detected boxes, class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []  # Handle as needed
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_idx]

            # Update unique track IDs
            if track_id not in unique_track_ids:
                unique_track_ids.add(track_id)

                # Update count for this class type
                if class_name in object_type_counts:
                    object_type_counts[class_name] += 1
                else:
                    object_type_counts[class_name] = 1

            # Update track history
            if track_id not in track_history:
                track_history[track_id] = []
            track_history[track_id].append((cx, cy))
            
            # Keep only the last 20 frames
            if len(track_history[track_id]) > max_frames:
                track_history[track_id].pop(0)
            
            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw the track history
            if len(track_history[track_id]) > 1:
                for i in range(1, len(track_history[track_id])):
                    cv2.line(frame, track_history[track_id][i-1], track_history[track_id][i], (255, 0, 0), 2)

        # Display counts on the frame
        total_objects = len(unique_track_ids)
        cv2.putText(frame, f"Total Objects: {total_objects}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y_offset = 50
        for class_name, count in object_type_counts.items():
            cv2.putText(frame, f"{class_name}: {count}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            y_offset += 20

        cv2.imshow('yolo_tracking', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

# Initial confidence threshold
confidence_threshold = 50  # Percentage (0-100)

# Callback function for the trackbar
def update_confidence(x):
    global confidence_threshold
    confidence_threshold = x / 100  # Convert to decimal (0.0-1.0)

# Create a window
cv2.namedWindow('yolo_tracking')

# Create the trackbar
cv2.createTrackbar('Confidence', 'yolo_tracking', 50, 100, update_confidence)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking with the current confidence threshold
    results = model.track(frame, persist=True, conf=confidence_threshold)

    if results[0].boxes.data is not None:
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_idx]

            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display the current confidence threshold
    cv2.putText(frame, f"Confidence: {confidence_threshold:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('yolo_tracking', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch

# Initial confidence and NMS thresholds (in percentage)
confidence_threshold = 50  # Confidence threshold in percentage (0-100)
nms_threshold = 40  # NMS threshold in percentage (0-100)

# Callback function for the confidence trackbar
def update_confidence(x):
    global confidence_threshold
    confidence_threshold = x / 100  # Convert to decimal (0.0-1.0)

# Callback function for the NMS trackbar
def update_nms(x):
    global nms_threshold
    nms_threshold = x / 100  # Convert to decimal (0.0-1.0)

# Create a window for displaying the video and parameters
cv2.namedWindow('yolo_tracking')

# Create trackbars for confidence and NMS thresholds
cv2.createTrackbar('Confidence', 'yolo_tracking', 50, 100, update_confidence)
cv2.createTrackbar('NMS Threshold', 'yolo_tracking', 40, 100, update_nms)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking with current confidence and NMS thresholds
    results = model.track(frame, persist=True, conf=confidence_threshold)

    if results[0].boxes.data is not None:
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        # Apply NMS with the current NMS threshold
        indices = cv2.dnn.NMSBoxes(boxes.tolist(), confidences.tolist(), score_threshold=confidence_threshold, nms_threshold=nms_threshold)

        for i in indices.flatten():
            x1, y1, x2, y2 = map(int, boxes[i])
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_indices[i]]

            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_ids[i]} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display the current confidence and NMS threshold values
    cv2.putText(frame, f"Confidence: {confidence_threshold:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"NMS Threshold: {nms_threshold:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show the frame with the bounding boxes and tracking
    cv2.imshow('yolo_tracking', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch

# Initial confidence and NMS thresholds (in percentage)
confidence_threshold = 50  # Confidence threshold in percentage (0-100)
nms_threshold = 40  # NMS threshold in percentage (0-100)

# Callback function for the confidence trackbar
def update_confidence(x):
    global confidence_threshold
    confidence_threshold = x / 100  # Convert to decimal (0.0-1.0)

# Callback function for the NMS trackbar
def update_nms(x):
    global nms_threshold
    nms_threshold = x / 100  # Convert to decimal (0.0-1.0)

# Create a window for displaying the video and parameters
cv2.namedWindow('yolo_tracking')

# Create trackbars for confidence and NMS thresholds
cv2.createTrackbar('Confidence', 'yolo_tracking', 50, 100, update_confidence)
cv2.createTrackbar('NMS Threshold', 'yolo_tracking', 40, 100, update_nms)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking with current confidence and NMS thresholds
    results = model.track(frame, persist=True, conf=confidence_threshold)

    if results[0].boxes.data is not None:
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        # Apply NMS with the current NMS threshold
        indices = cv2.dnn.NMSBoxes(boxes.tolist(), confidences.tolist(), score_threshold=confidence_threshold, nms_threshold=nms_threshold)

        for i in indices.flatten():
            x1, y1, x2, y2 = map(int, boxes[i])
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_indices[i]]

            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_ids[i]} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display the current confidence and NMS threshold values
    cv2.putText(frame, f"Confidence: {confidence_threshold:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"NMS Threshold: {nms_threshold:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show the frame with the bounding boxes and tracking
    cv2.imshow('yolo_tracking', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Dictionary to store center points for each track_id
track_history = {}

# Set the maximum number of frames to track
max_frames = 20

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True , classes = [0] )
    
    if results[0].boxes.data is not None:
        # Get detected boxes, class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []  # Handle as needed
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_idx]

            # Update track history
            if track_id not in track_history:
                track_history[track_id] = []
            track_history[track_id].append((cx, cy))
            
            # Keep only the last 20 frames
            if len(track_history[track_id]) > max_frames:
                track_history[track_id].pop(0)
            
            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw the track history
            if len(track_history[track_id]) > 1:
                for i in range(1, len(track_history[track_id])):
                    cv2.line(frame, track_history[track_id][i-1], track_history[track_id][i], (255, 0, 0), 2)

        cv2.imshow('yolo_tracking', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
max_frames = 30
# Define horizontal line positions
entry_line_y = 100
exit_line_y = 600

# Counters for entries and exits
entry_count = 0
exit_count = 0

# Add the following inside your while loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Draw horizontal lines
    cv2.line(frame, (0, entry_line_y), (frame.shape[1], entry_line_y), (0, 255, 0), 2)  # Entry line
    cv2.line(frame, (0, exit_line_y), (frame.shape[1], exit_line_y), (0, 0, 255), 2)    # Exit line

    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True, classes=[0])

    if results[0].boxes.data is not None:
        boxes = results[0].boxes.xyxy.cpu()
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = []
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            class_name = class_names[class_idx]

            # Update track history
            if track_id not in track_history:
                track_history[track_id] = []
            track_history[track_id].append((cx, cy))

            if len(track_history[track_id]) > max_frames:
                track_history[track_id].pop(0)

            # Check crossing entry and exit lines
            if len(track_history[track_id]) > 1:
                prev_cy = track_history[track_id][-2][1]
                
                # Check entry
                if prev_cy < entry_line_y <= cy:
                    entry_count += 1
                    print(f"Entry detected: ID {track_id}")

                # Check exit
                if prev_cy > exit_line_y >= cy:
                    exit_count += 1
                    print(f"Exit detected: ID {track_id}")

            # Draw center point and track
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw track history
            if len(track_history[track_id]) > 1:
                for i in range(1, len(track_history[track_id])):
                    cv2.line(frame, track_history[track_id][i-1], track_history[track_id][i], (255, 0, 0), 2)

    # Display entry/exit counts
    cv2.putText(frame, f"Entries: {entry_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Exits: {exit_count}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('yolo_tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Train the model
train_results = model.train(
    data=r"G:\number_plate\data.yaml",  
    epochs=10, 
    imgsz=640,  
    device="cpu",  
)

# Evaluate model performance on the validation set
metrics = model.val()



In [ ]:

# Export the model to ONNX format
path = model.export(format="onnx")  